In [1]:
import pandas as pd
import os

In [3]:
data_raw = pd.read_csv('../data/data_raw.csv', dtype=object, index_col=False)

In [4]:
data_index = data_raw.drop('Unnamed: 0', axis=1)
data_index.set_index('ride_id', inplace=True)

In [6]:
data_index.head(10)

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
ride_id,,,,,,,,,,,,
A847FADBBC638E45,docked_bike,2020-04-26 17:45:14,2020-04-26 18:12:03,Eckhart Park,86,Lincoln Ave & Diversey Pkwy,152,41.8964,-87.661,41.9322,-87.6586,member
5405B80E996FF60D,docked_bike,2020-04-17 17:08:54,2020-04-17 17:17:03,Drake Ave & Fullerton Ave,503,Kosciuszko Park,499,41.9244,-87.7154,41.9306,-87.7238,member
5DD24A79A4E006F4,docked_bike,2020-04-01 17:54:13,2020-04-01 18:08:36,McClurg Ct & Erie St,142,Indiana Ave & Roosevelt Rd,255,41.8945,-87.6179,41.8679,-87.623,member
2A59BBDF5CDBA725,docked_bike,2020-04-07 12:50:19,2020-04-07 13:02:31,California Ave & Division St,216,Wood St & Augusta Blvd,657,41.903,-87.6975,41.8992,-87.6722,member
27AD306C119C6158,docked_bike,2020-04-18 10:22:59,2020-04-18 11:15:54,Rush St & Hubbard St,125,Sheridan Rd & Lawrence Ave,323,41.8902,-87.6262,41.9695,-87.6547,casual
356216E875132F61,docked_bike,2020-04-30 17:55:47,2020-04-30 18:01:11,Mies van der Rohe Way & Chicago Ave,173,Streeter Dr & Grand Ave,35,41.8969,-87.6217,41.8923,-87.612,member
A2759CB06A81F2BC,docked_bike,2020-04-02 14:47:19,2020-04-02 14:52:32,Streeter Dr & Grand Ave,35,Fairbanks St & Superior St,635,41.8923,-87.612,41.8957,-87.6201,member
FC8BC2E2D54F35ED,docked_bike,2020-04-07 12:22:20,2020-04-07 13:38:09,Ogden Ave & Roosevelt Rd,434,Western Ave & Congress Pkwy,382,41.8665,-87.6847,41.8747,-87.6864,casual
9EC5648678DE06E6,docked_bike,2020-04-15 10:30:11,2020-04-15 10:35:55,LaSalle Dr & Huron St,627,Larrabee St & Division St,359,41.8949,-87.6323,41.9035,-87.6434,casual


In [7]:
data_index.tail(10)

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
ride_id,,,,,,,,,,,,
4A927B42ACC8A5A7,electric_bike,2023-05-15 08:50:43,2023-05-15 08:58:29,Columbus Dr & Randolph St,13263,NaN,NaN,41.88469398,-87.619663358,41.89,-87.64,member
2A91830F1F4C64FB,electric_bike,2023-05-04 11:25:33,2023-05-04 19:25:27,Ellis Ave & 55th St,KA1504000076,NaN,NaN,41.794359446,-87.60145402,41.8,-87.6,member
EB268324951C550D,electric_bike,2023-05-31 08:48:50,2023-05-31 08:59:17,Columbus Dr & Randolph St,13263,NaN,NaN,41.88449,-87.61931516666667,41.88,-87.64,member
D205FEAFFFA7899F,electric_bike,2023-05-23 20:43:30,2023-05-23 21:07:22,Columbus Dr & Randolph St,13263,NaN,NaN,41.88469766666667,-87.6193755,41.88,-87.65,member
DC8A39752C15ACF5,electric_bike,2023-05-01 22:25:50,2023-05-01 22:40:53,Clark St & Elmdale Ave,KA1504000148,NaN,NaN,41.990883833333335,-87.66957433333333,41.97,-87.65,member
48BDA26F34445546,electric_bike,2023-05-18 10:26:43,2023-05-18 10:48:00,Clark St & Elmdale Ave,KA1504000148,NaN,NaN,41.990875602,-87.669721484,42.0,-87.66,member
573025E5EDE10DE1,electric_bike,2023-05-17 14:32:48,2023-05-17 14:45:37,State St & 33rd St,13216,NaN,NaN,41.834733963,-87.625798345,41.83,-87.62,member
D88D48898C6FB63E,electric_bike,2023-05-17 07:59:29,2023-05-17 08:04:54,Columbus Dr & Randolph St,13263,NaN,NaN,41.884421706,-87.619392514,41.88,-87.63,member
4692DCD2F87497F5,electric_bike,2023-05-18 08:34:48,2023-05-18 08:38:40,Public Rack - Karlov Ave & Lawrence Ave,1127.0,NaN,NaN,41.97,-87.73,41.97,-87.74,member
